# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

In [63]:
'''
Your Code
'''
# 打開瀏覽器
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")
# 模擬使用者操作行為，選擇/點擊
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
# selectSite.select_by_visible_text("臺北市-士林")
selectSite.select_by_value('11')

selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

time.sleep(2)

In [64]:
# 取得資料，丟到 BeautifulSoup 解析
html_source = browser.page_source
soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')
#<table class="TABLE_G" cellspacing="0" cellpadding="3" align="Center" rules="all" border="1" 
#    id="ctl05_gv" style="border-color:Black;border-width:1px;border-style:None;border-collapse:collapse;">

th = table.find_all('th')
#<th scope="col">監測項目</th><th scope="col">單位</th><th scope="col">監測日期</th><th scope="col">監測值</th><th scope="col">標註</th>
columns = [tag.text for tag in th]
# print(columns) #==['監測項目', '單位', '監測日期', '監測值', '標註']

# <tr style="color:Black;">
# 	<td class="no-alt" rowspan="12" valign="top" style="white-space:nowrap;">SO2</td>
# 	<td class="no-alt" rowspan="12" valign="top">ppb</td>
# 	<td>2018/01</td><td>1.80</td>
# 	<td>&nbsp;</td>
# </tr>

# <tr class="ALT" style="color:Black;">
# 	<td>2018/02</td>
# 	<td>1.90</td>
# 	<td>&nbsp;</td>
# </tr>

d = {} #字典(Dictionary)
td = table.find_all('td')

# <td class="no-alt" rowspan="12" style="white-space:nowrap;" valign="top">SO2</td>
# <td class="no-alt" rowspan="12" valign="top">ppb</td>
# <td>2018/01</td>
# <td>1.80</td>
# <td> </td>
for tag in td:
#     print(tag)
#     print("--------")
    if tag.attrs and tag.get('style'): #有標題 SO2 ,attrs参数是dict类型
        d.setdefault(tag.text, {}) #放 空格
        #setdefault=如果鍵不存在於字典中，將會添加鍵並將值設為默認值。,,
            #dict . setdefault ( key , default = None ) 
            #key -- 查找的鍵值。
            #default -- 鍵不存在時，設置的默認鍵值。        
    elif not tag.attrs and tag.text.strip(): #沒標題 SO2
        #strip() 方法用於移除字符串頭尾指定的字符（默認為空格或換行符）或字符序列。
        #注意：該方法只能刪除開頭或是結尾的字符，不能刪除中間部分的字符。
        if '/' in tag.text: #2018/02
            k = tag.text
#             print("k ",k)#==
        else:
            d[list(d.keys())[-1]].setdefault(k, tag.text) #監測值
            #列表(List),
            #keys() 函数以列表返回一个字典所有的键。,
                #dict = {'Name': 'Zara', 'Age': 7}
                #print "Value : %s" %  dict.keys()
                #>>>Value : ['Age', 'Name']
#             print( d[list(d.keys())[-1]].setdefault(k, tag.text))#==
#     print(d)
#     print("--------")#==
# d
df = pd.DataFrame(d)
df2 = df[['SO2']].iloc[:8].reset_index() #取出SO2
df2.columns = columns[2:4] #放標題
df2

,監測日期,監測值
0,2018/01,1.80
1,2018/02,1.90
2,2018/03,2.20
3,2018/04,2.30
4,2018/05,3.10
5,2018/06,2.70
6,2018/07,2.20
7,2018/08,2.40


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [65]:
'''
Your Code
'''
#----1-----------------------
df = pd.DataFrame(d)
# print(columns) #==
# print(df) #==
df2 = df[['SO2', 'CO']].iloc[:8].reset_index() #iloc[:8]=y軸1~8月
#reset_index可以還原索引，重新變為默認的整型索引 
# print(df2) #==
old_columns = list(df2.columns) #['index', 'SO2', 'CO']
# print(old_columns[1:]) #==
new_columns = columns[2:3] + \
              list(map(lambda x: str(x) + '-' + ''.join(columns[3:4]), old_columns[1:])) #['SO2-監測值', 'CO-監測值']
#columns=['監測項目', '單位', '監測日期', '監測值', '標註']
# columns[2:3]=['監測日期'],columns[3:4]=['監測值'],old_columns[1:]=['SO2', 'CO']

#map()會根據提供的函數對指定序列做映射
# listA = ['1','2','3'] #意思就是將list裡面的每一個元素都轉型成整數(int)
# print map(int,listA)
# [1, 2, 3]

# join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串。
# str = "-";
# seq = ("a", "b", "c"); # 字符串序列
# print str.join( seq )
# >>>a-b-c

# print(''.join(columns[3:4]), old_columns[1:])
# >>> 監測值 ['SO2', 'CO']

# print(lambda x: str(x) + '-' + ''.join(columns[3:4]), old_columns[1:])
# >>> <function <lambda> at 0x00000284A98ED6A8> ['SO2', 'CO']

df2.columns = new_columns
df2

# #----2----------------
# d = {}
# key = ''
# date = ''
# items = table.find_all('td')
# for item in items:
# #     print(item)
#     if 'style' in item.attrs:
#         key = item.text
#         if key in ['SO2','CO']:
#             d[key]={}
#     elif 'class' not in item.attrs:
#         if item.text.strip() != '':
# #             print(item.text)
#             if '/'in item.text:
#                 date = item.text
#             else:
# #                 print(item.text)
#                 if key in ['SO2','CO'] and date <= '2018/08':
#                     d[key][date] = item.text
# pd.DataFrame(d)         

# #---3---------------------------  
# table = soup.find(class_='TABLE_G')
# data = dict()
# for tr in table.find_all('tr')[1:]:
#     td = tr.find_all('td')
#     if len(td) == 5:
#         Name = td[0].text
#         data[Name] = {}
#         Date = td[2].text
#         Value = td[3].text
#         data[Name][Date] = Value
#     if len(td) == 3:
#         Date = td[0].text
#         Value = td[1].text
#         data[Name][Date] = Value
        
# result = pd.DataFrame(data)
# print(result)
# print(result[['SO2']][:8])
# print(result[['SO2','CO']][:8])

,監測日期,SO2-監測值,CO-監測值
0,2018/01,1.80,0.34
1,2018/02,1.90,0.44
2,2018/03,2.20,0.40
3,2018/04,2.30,0.38
4,2018/05,3.10,0.34
5,2018/06,2.70,0.29
6,2018/07,2.20,0.21
7,2018/08,2.40,0.30


In [66]:
# 關閉瀏覽器
browser.quit();

In [57]:

# print(lambda x: str(x) + '-' + ''.join(columns[3:4]), old_columns[1:])
# print(list(map(lambda x: str(x) + '-' + ''.join(columns[3:4]), old_columns[1:])))
print(list)

[]
